# Take an On Demand Log Backup of a MSSQL Database
Like an On Demand Snapshot, DBAs have the need to initiate an on demand log backup. When taking the log backup, all that is required is the database information, as log backups are only retained as long as defined in the SLA Domain that is assigned to the database. 

# Workflow
To get information about the SQL Server Instance, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **New-RscMssqlLogBackup** to initiate an On Demand Log Backup of a database. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC
 new-rscmssqllogBackup -RscMssqlDatabase $mssqldatabase  

## Connect to RSC

In [1]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC


Loading Rubrik Security Cloud PowerShell Module (PowerShell Core)...

Welcome to the Rubrik Security Cloud PowerShell SDK!

	Please refer to the Getting Started section of the Readme documentation: 
	https://github.com/rubrikinc/rubrik-powershell-sdk#getting-started


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20230920-45     v20231018-24



## Get the Cluster ID

In [3]:
$ClusterName = "vault-r-london"
$RubrikCluster = Get-RscCluster -Name $ClusterName

Get-RscCluster field profile: DEFAULT


## Get the Instance Information

In [4]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscMssqlInstance = Get-RscMssqlInstance -HostName $HostName  -ClusterId $RubrikCluster.id

Get-RscMssqlInstance field profile: DEFAULT


## Get the Database Information

In [6]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance

Get-RscMssqlDatabase field profile: DEFAULT


## Initiate On Demand Log Backup

In [8]:
New-RscMssqlLogBackup -RscMssqlDatabase $RscMssqlDatabase  


EndTime   : 
Id        : MSSQL_LOG_BACKUP_138e0fbc-c0f8-4df2-97b4-3338258e49c1_cbfe844c-4130-4635-bc72-cb98cc64a
            e19:::0
NodeId    : 
Progress  : 
StartTime : 
Status    : QUEUED
Error     : 
Links     : 




## Track the status of the backup

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc